In [ ]:
%matplotlib inline
import matplotlib as mpl
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import minmax_scale
from sklearn.decomposition import PCA


In [ ]:
#carrega o dataset
dados = pd.read_csv('/home/erasmor/Downloads/2017/MachineLearningCVE/2017_todas_classes.csv',sep=",",encoding = 'utf-8',  header=0,na_values='.',dtype={'Label':'category'})
#remove valores infinitos
dados.replace(-np.Inf, np.nan)
#substitui valores NaN
dados.fillna(dados.mean())


In [ ]:
#mostra como está a base de dados
dados.dtypes

In [ ]:
dados.memory_usage(deep=True)

In [ ]:
# verifica quantas instâncias (linhas) e quantos atributos (colunas) a base de dados contém
print("numero de linhas e colunas: ",dados.shape)

In [ ]:
#visualizar distribições por classes contidas no csv - informar nome da classe alvo
print(dados.groupby('Label').size())

In [ ]:
#X_raw_normalize = MinMaxScaler(X_raw_normalize.reshape(0, 1)).reshape(len(X_raw_normalize))
#X_raw_normalizetd2 = (X_raw_normalize - X_raw_normalize.min(axis=0)) / (X_raw_normalize.max(axis=0) - X_raw_normalize.min(axis=0))
# Obtendo os nomes das colunas do DataFrame como uma lista.
cols = list(dados.columns)
# colunas que nao serao normalizadas
cols.remove('Label')
# Copiando os dados e aplicando a normalizacao por reescala nas colunas do DataFrame que contem
# valores continuos. Por padrao, o metodo minmax_scale reescala com min=0 e max=1.
dados = dados[~dados.isin([np.nan, np.inf, -np.inf]).any(1)]
dados[cols] = dados[cols].apply(minmax_scale)


In [ ]:
#define as colunas de atributos e a coluna da classe (de 0 a 72 são atributos e após a 72 é a classe)
# "X_raw" é features/atributos e "y_raw" é target/classe ==> As duas formas abaixo dão certo.
#array = dataset.values
#X_raw = array[:,0:72]
#y_raw = array[:,72]
X_raw = dados.iloc[:, :-1].values # atributos
y_raw = dados.iloc[:, 78].values # classe de ataques
X_raw = np.nan_to_num(X_raw.astype(np.float32))


In [ ]:
#transformar a variável Y com valores categóricos das classses de ataques em valores:
labelencoder_y = LabelEncoder()
y_raw = labelencoder_y.fit_transform(y_raw)


In [ ]:
# Instanciando um PCA. O parametro n_components indica a quantidade de dimensoes que a base
# original sera reduzida.
pca = PCA(n_components=2, whiten=True,random_state=42)

In [ ]:
# Aplicando o pca na base de dados. O atributo 'values' retorna um numpy.array
# de duas dimensões (matriz) contendo apenas os valores numericos do DataFrame.
X_raw = pca.fit_transform(X_raw)


In [ ]:
def treino(X_raw, y_raw, idx_data, idx_dobra, TRAIN, TEST, classifier):
    
    from sklearn.neural_network import MLPClassifier
    from sklearn.ensemble import RandomForestClassifier
    
    from xgboost import XGBClassifier
    from sklearn import svm
    from sklearn.svm import LinearSVC
    from sklearn.datasets import make_hastie_10_2
    from sklearn.ensemble import GradientBoostingClassifier
    from sklearn.tree import DecisionTreeClassifier
    from sklearn.naive_bayes import GaussianNB
    from sklearn.neighbors import KNeighborsClassifier
    from sklearn.metrics import precision_score, recall_score, confusion_matrix, classification_report, accuracy_score, f1_score
    import functools
    
    #define nome de arquivos para salvar
    indica_pool=str(idx_dobra)
        
    # recupera amostras de teste de acordo com a dobra em uso
    X_test, y_test = X_raw[idx_data[idx_dobra][TEST]], y_raw[idx_data[idx_dobra][TEST]]
    # recupera amostras de treino de acordo com a dobra em uso
    X_train, y_train = X_raw[idx_data[idx_dobra][TRAIN]], y_raw[idx_data[idx_dobra][TRAIN]]
    
    #instancia classificadores de aprendizado
    if classifier == "knn":
        learner = KNeighborsClassifier(n_neighbors=5)
        arquivo_performance = open("baseline_performance_knn_dobra_"+indica_pool+".txt","a")
                
    if classifier == "rf":
        learner = RandomForestClassifier(random_state=42)
        arquivo_performance = open("baseline_performance_rf_dobra_"+indica_pool+".txt","a")
                
    if classifier == "nb":
        learner = GaussianNB()
        arquivo_performance = open("baseline_performance_nb_dobra_"+indica_pool+".txt","a")
                
    if classifier == "tree":
        learner = DecisionTreeClassifier()
        arquivo_performance = open("baseline_performance_tree_dobra_"+indica_pool+".txt","a")
                
    if classifier == "mlp":
        learner = MLPClassifier(max_iter=2000)
        arquivo_performance = open("baseline_performance_mlp_dobra_"+indica_pool+".txt","a")
            
    if classifier == "xgb":
        learner = GradientBoostingClassifier(random_state=42)
        arquivo_performance = open("baseline_performance_xgb_dobra_"+indica_pool+".txt","a")
        
    if classifier == "svm":
        learner = svm.SVC(kernel='linear',probability=True)
        arquivo_performance = open("baseline_performance_svm_dobra_"+indica_pool+".txt","a")
        
      
    # treina o modelo
    learner.fit(X_train, y_train)
    predictions=learner.predict(X_test)
        
    #print('Accuracy: %f' % accuracy_score(y_test,predictions))
    arquivo_performance.write('Accuracy %f \n' % accuracy_score(y_test,predictions))
    #print ('Precision:  %f' % precision_score(y_test, predictions,average='macro',zero_division=1))
    arquivo_performance.write('Precision: %f \n' % precision_score(y_test, predictions,average='macro',zero_division=1))
    #print ('F1 score: %f' % f1_score(y_test, predictions,average='macro',zero_division=1))
    arquivo_performance.write('Recall: %f \n' % recall_score(y_test, predictions, average='macro',zero_division=1))
    #arquivo_performance.write('F1 score: %f \n' % f1_score(y_test, predictions,average='macro',zero_division=1))
    #print ('Recall: %f' % recall_score(y_test, predictions, average='macro',zero_division=1))
    f1score= 2*((precision_score(y_test, predictions,average='macro',zero_division=1)*recall_score(y_test, predictions, average='macro',zero_division=1))/(precision_score(y_test, predictions,average='macro',zero_division=1)+recall_score(y_test, predictions, average='macro',zero_division=1)))
    arquivo_performance.write('F1 score: %f \n' % f1score)
    #print("========================================")
    arquivo_performance.write("\n Avaliação por classe \n")
    arquivo_performance.write(classification_report(y_test, predictions,zero_division=1))
    arquivo_performance.close()

    
      

import time
import sys
import threading
from datetime import datetime
from datetime import date

from sklearn.model_selection import StratifiedShuffleSplit, ShuffleSplit, train_test_split
from modAL.uncertainty import classifier_uncertainty
from modAL.models import ActiveLearner
from modAL.uncertainty import uncertainty_sampling
from timeit import Timer
import time
import functools

#inicia relogio
t1 = time.time()
# Define o tamanho das divisões feitas no dataset (cross-validation)
n_splits = 10

#define array de indices das partições
idx_data =[]
# cross validation bags - n_splits
data_cv = StratifiedShuffleSplit(n_splits= n_splits, train_size=350,test_size=350, random_state=42) 
data_cv.get_n_splits(X_raw, y_raw)
# chame a instância e gere os dados sobre a base original
type(data_cv.split(X_raw, y_raw))
# dividir os dados - A função split.split () retorna índices para amostras de treino e amostras de teste. 
# Ele examinará o número de validação cruzada especificado e retornará cada vez que treinar 
# e testar os índices de amostra usando os conjuntos de dados de treinamento e teste que podem 
# ser criados filtrando o conjunto de dados inteiro. Por exemplo idx_data[0][1], o primeiro indice faz referencia
# a dobra e o segundo indice faz referencia a posição da dobra (0 = treino e 1 = teste)
for train_index, test_index in data_cv.split(X_raw,y_raw):
    #print("TRAIN:", train_index, "TEST:", test_index)
    #print("n_split",n_splits,"TRAIN:", train_index, "TEST:", test_index)
    idx_data.append([train_index, test_index])
#verifica tamanho das dobras (numero de instâncias de cada dobra)
#print("tamanho de cada dobra: ",idx_data[3][0].shape)

TRAIN =0
TEST =1
#chama procedimento de aprendizado para todas as dobras

classificador_knn="knn"
classificador_rf="rf"
classificador_nb="nb"
classificador_tree="tree"
classificador_mlp="mlp"
classificador_xgb="xgb"
classificador_svm="svm"

for idx_dobra in range(n_dobras):    
    threading.Thread(target=treino(X_raw, y_raw, idx_data, idx_dobra, TRAIN, TEST, classificador_knn)).start()
    threading.Thread(target=treino(X_raw, y_raw, idx_data, idx_dobra, TRAIN, TEST, classificador_rf)).start()
    threading.Thread(target=treino(X_raw, y_raw, idx_data, idx_dobra, TRAIN, TEST, classificador_nb)).start()
    threading.Thread(target=treino(X_raw, y_raw, idx_data, idx_dobra, TRAIN, TEST, classificador_mlp)).start()
    threading.Thread(target=treino(X_raw, y_raw, idx_data, idx_dobra, TRAIN, TEST, classificador_xgb)).start()
    threading.Thread(target=treino(X_raw, y_raw, idx_data, idx_dobra, TRAIN, TEST, classificador_tree)).start()
    threading.Thread(target=treino(X_raw, y_raw, idx_data, idx_dobra, TRAIN, TEST, classificador_svm)).start()
    
    
t2 = time.time()
time_elapsed = (t2-t1)/3600
print("tempo: ",time_elapsed)